In [1]:
!pip install sdv synthcity
!pip install opacus==1.4.0

In [2]:
!unrar x data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from data.rar


Would you like to replace the existing file data/moons_dataset.csv
120298 bytes, modified on 2025-06-02 20:50
with a new one
120298 bytes, modified on 2025-06-02 20:50

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit Q

Program aborted


In [14]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from synthcity.plugins import Plugins
from synthcity.plugins.generic.plugin_ddpm import TabDDPMPlugin as DDPM
from sdv.metadata import SingleTableMetadata
from xgboost import XGBClassifier
from hyperopt import space_eval
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor

In [15]:
dir_datasets = 'data/'

# Загрузка реальных датасетов
# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'moons_dataset.csv')

# Словарь датасетов для удобства
datasets = {
    'two_moons': {
                    "data": real_data_1,
                    "task": "regression",
                    "target": "x2",
                    "num_columns":
                    ["x1"],
                    "cat_columns":
                    []
                },
}


for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f"Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете two_moons:
 Количество строк: 3000
 Количество колонок: 2
 Колонки: ['x1', 'x2']
 Задача: regression
 Целевая переменная: x2
 Числовые признаки: ['x1']
Категориальные признаки: []



In [16]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [17]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [18]:
# Определение пространства поиска гиперпараметров для DDPM
ddpm_space = {
    'batch_size': hp.choice('batch_size', [256, 4096]),
    'lr': hp.qloguniform('lr', np.log(1e-4), np.log(1e-3), 1e-5),
    'num_timesteps': hp.choice('num_timesteps', [1000]),
    'n_layers_hidden': hp.choice('n_layers_hidden', [2, 4, 6, 8]),
    'n_units_hidden': hp.choice('n_units_hidden', [128, 256, 512, 1024]),
    'n_iter': hp.choice('n_iter', [20000]),
    'transformation_num_type': hp.choice('transformation_num_type', ['None']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['None'])
}

In [19]:
def evaluate_ml_efficacy(real_train, synthetic_train, real_test, target_column, num_columns, cat_columns, task_type='classification'):
    """
    Вычисляет ML-Efficacy метрику

    Args:
        real_train: реальные тренировочные данные
        synthetic_train: синтетические тренировочные данные
        real_test: реальные тестовые данные
        target_column: название целевой колонки
        task_type: 'classification' или 'regression'

    Returns:
        ml_efficacy_score: отношение качества модели на синтетических данных к качеству на реальных
    """

    # Подготовка данных
    real_train_copy = real_train.copy()

    # Обработка синтетических данных (если это объект с методом dataframe)
    if hasattr(synthetic_train, "dataframe"):
        synthetic_train = synthetic_train.dataframe()
    synthetic_train_copy = synthetic_train.copy()

    real_test_copy = real_test.copy()

    # Определение категориальных колонок (исключая целевую)
    feature_columns = num_columns
    categorical_columns = cat_columns

    # Label Encoding для категориальных признаков
    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        # Обучаем на реальных данных
        combined_values = pd.concat([real_train_copy[col], real_test_copy[col]]).astype(str)
        le.fit(combined_values)
        label_encoders[col] = le

        # Применяем к реальным данным
        real_train_copy[col] = le.transform(real_train_copy[col].astype(str))
        real_test_copy[col] = le.transform(real_test_copy[col].astype(str))

        # Применяем к синтетическим данным, обрабатываем неизвестные значения
        synthetic_values = synthetic_train_copy[col].astype(str)
        known_values = set(le.classes_)
        synthetic_values = synthetic_values.apply(lambda x: x if x in known_values else le.classes_[0])
        synthetic_train_copy[col] = le.transform(synthetic_values)

    # Разделение на признаки и целевую переменную
    X_real_train = real_train_copy.drop(columns=[target_column])
    y_real_train = real_train_copy[target_column]

    X_synthetic_train = synthetic_train_copy.drop(columns=[target_column])
    y_synthetic_train = synthetic_train_copy[target_column]

    X_real_test = real_test_copy.drop(columns=[target_column])
    y_real_test = real_test_copy[target_column]

    # Выбор модели в зависимости от типа задачи
    if task_type == 'classification':
        model_real = XGBClassifier(random_state=42, eval_metric='logloss')
        model_synthetic = XGBClassifier(random_state=42, eval_metric='logloss')
        score_func = f1_score
        score_kwargs = {'average': 'weighted'} if len(np.unique(y_real_train)) > 2 else {'average': 'binary'}
    else:  # regression
        model_real = XGBRegressor(random_state=42)
        model_synthetic = XGBRegressor(random_state=42)
        score_func = r2_score
        score_kwargs = {}

    try:
        # Обучение модели на реальных данных
        model_real.fit(X_real_train, y_real_train)
        y_pred_real = model_real.predict(X_real_test)
        score_real = score_func(y_real_test, y_pred_real, **score_kwargs)

        # Обучение модели на синтетических данных
        model_synthetic.fit(X_synthetic_train, y_synthetic_train)
        y_pred_synthetic = model_synthetic.predict(X_real_test)
        score_synthetic = score_func(y_real_test, y_pred_synthetic, **score_kwargs)

        # Вычисление ML-Efficacy
        if score_real > 0:
            ml_efficacy = score_synthetic / score_real
        else:
            ml_efficacy = 0.0

        return ml_efficacy

    except Exception as e:
        print(f"Ошибка при вычислении ML-Efficacy: {e}")
        return 0.0

In [20]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [21]:
from sklearn.model_selection import KFold

def generate_and_evaluate_ml_efficacy(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # Извлекаем информацию о задаче и целевой переменной из dataset_info
    task_type = dataset_info['task']
    target_column = dataset_info['target']

    data = process_data(dataset_info['data'], dataset_info['num_columns'], params['transformation_num_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    ml_efficacy_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(data=train_data)

        # Создание и обучение DDPM с заданными параметрами
        ddpm = DDPM(
            lr=params['lr'],
            num_timesteps=params['num_timesteps'],
            model_params = dict(n_layers_hidden=params['n_layers_hidden'],
                                n_units_hidden=params['n_units_hidden'],
                                dropout=0.0),
            #n_iter=params['n_iter'],
            batch_size=params['batch_size']
        )

        # Обучение модели на реальных данных
        with suppress_all_output():
            ddpm.fit(train_data)

        # Генерация синтетических данных
        synthetic_data = ddpm.generate(len(train_data))

        # Вычисляем ML-Efficacy
        score = evaluate_ml_efficacy(
            real_train=train_data,
            synthetic_train=synthetic_data,
            real_test=test_data,
            target_column=target_column,
            num_columns = dataset_info["num_columns"],
            cat_columns = dataset_info["cat_columns"],
            task_type=task_type
        )
        ml_efficacy_scores.append(score)

    return np.mean(ml_efficacy_scores)

In [22]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        ml_efficacy_score = generate_and_evaluate_ml_efficacy(params, dataset_info)

        print(f"ML-Efficacy: {ml_efficacy_score}")
        print(params)
        print("="*50)

        # Для ML-Efficacy мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -ml_efficacy_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'ml_efficacy_score': ml_efficacy_score
        }

    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(42)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ddpm_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ddpm_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_ml_efficacy = best_trial['result'].get('ml_efficacy_score', None)

    results = {
        'best_params': best_params,
        'best_ml_efficacy': best_ml_efficacy,
        'ml_efficacy_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший ML-Efficacy для {dataset_name}: {best_ml_efficacy}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [ ]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 100

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: two_moons
ML-Efficacy: 0.9250147810052316
{'batch_size': 256, 'lr': 0.00042, 'n_iter': 20000, 'n_layers_hidden': 8, 'n_units_hidden': 128, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
ML-Efficacy: 0.9223971703247992
{'batch_size': 4096, 'lr': 0.00041000000000000005, 'n_iter': 20000, 'n_layers_hidden': 4, 'n_units_hidden': 1024, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
ML-Efficacy: 0.8818160878078407
{'batch_size': 256, 'lr': 0.00012000000000000002, 'n_iter': 20000, 'n_layers_hidden': 2, 'n_units_hidden': 128, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
ML-Efficacy: 0.8949646636451493
{'batch_size': 4096, 'lr': 0.00016, 'n_iter': 20000, 'n_layers_hidden': 4, 'n_units_hidden': 512, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
ML-Efficacy: 0.914877449850637
{'batch_size

In [ ]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_ml_efficacy'])

ML-Efficacy: 0.9826928199330767<br>
{'batch_size': 256, 'lr': 0.00059, 'n_iter': 20000, 'n_layers_hidden': 6, 'n_units_hidden': 128, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}